# Diseño de acople sintonizado

* Parametros
* Fijar CT >> 10 4pF
* Con fo sacar Lteo
* Diseño y verificacion con curvas K/J
* Fijar L, obtener su r y ajustar CT
* Con el factor Q sacar RT
* Con rrRa = 2RT, sacar relacion C2/C1
* Con rrRL//r = 2RT, sacar relacion C4/C3

In [81]:
import sympy as sp
from sympy.core.compatibility import iterable
import numpy as np

# Funciones auxiliares
def mem2mem(equation,opp):
    """mem2mem:
        Aplica una operacion sencilla en ambos miembros de una ecuacion.

    Args:
        equation: Ecuacion a operar.
        opp: Termino para operar en ambos miembros.

    Returns: 
        Ecuacion de resultado luego de operar.
    """
    sel=opp[0]
    expr=sp.sympify(opp[1::])
    
    if sel=="+":
        return sp.Eq(equation.lhs+expr,equation.rhs+expr)
    elif sel=="-":
        return sp.Eq(equation.lhs-expr,equation.rhs-expr)
    elif sel=="*":
        return sp.Eq(equation.lhs*expr,equation.rhs*expr)
    elif sel=="/":
        return sp.Eq(equation.lhs/expr,equation.rhs/expr)

def disp(ecuaciones,titulo=""):
    """disp:
        Mostrar una ecuacion con titulo y separadores

    Args:
        ecuaciones: Ecuacion para imprimir
        titulo: Titulo opcional a mostrar. Defaults to "".
    """
    print(titulo + "\n")
    
    if type(ecuaciones)!=iterable:
        sp.pprint(ecuaciones,num_columns=200)
    
    else:
        for ecc in ecuaciones:
            sp.pprint(ecc,num_columns=200)

    print("\n" + "-"*80)

## Parametros

In [79]:
# frec de resonancia, 1(ultimoNumDNI) MHz
fo=16E6
# ancho de banda, 10%fo
BW=10*fo/100
# impedancias
Ra=50
RL=1E3

# como CT >> 10*4pF, fijar CT=25*4pF=100pF
veces=25
CT=4E-12*veces

print(f"requerimientos: \n\tfrec de resonancia = {fo/1E6} MHz, BW = {BW/1E6} MHz \n\timpedancias entrada/salida {Ra} Ohm/{RL/1E3} kOhm\n")
print(f"se adopta un CT de {CT/1E-12} pF ({veces} veces 4pF)")

requerimientos: 
	frec de resonancia = 16.0 MHz, BW = 1.6 MHz 
	impedancias entrada/salida 50 Ohm/1.0 kOhm

se adopta un CT de 100.0 pF (25 veces 4pF)


## Sacar L (teorico)

In [80]:
# frec de resonancia
""" fo = 1/(2*pi * sqrt(LT*CT)) """
ssfo,ssLT,ssCT=sp.symbols("ssfo,ssLT,ssCT")
ec0=sp.Eq(ssfo,1/(2*sp.pi*sp.sqrt(ssLT*ssCT)))
disp(ec0,"Frecuencia de resonancia:")

print("resolviendo para LT")
print(sp.solve(ec0,ssLT))

# despejando LT
""" 1/(4 * pi^2 * ssCo * ssfo^2) """

LT=1/(4*np.pi**2 * CT * fo**2)
print(f"\nL(teorico) = {LT/1E-6} uH.")

Frecuencia de resonancia:

               1        
ssfo = ─────────────────
             ___________
       2⋅π⋅╲╱ ssCT⋅ssLT 

--------------------------------------------------------------------------------
resolviendo para LT
[1/(4*pi**2*ssCT*ssfo**2)]

L(teorico) = 0.9894646840072047 uH.


## Diseño del inductor (curvas J/K)

In [77]:
# redondear inductancia [uH]
LT=0.1

# diametro del cobre 1mm
Dcu=1E-3
# diametro del nucleo de aire 1cm
Dnuc=1E-2

# diametro total del inductor [cm]
Dind=(Dnuc+Dcu)/1E-2

print(f"para inductor de {LT} uH, con nucleo de {Dnuc/1E-2} cm y cobre de {Dcu/1E-3} mm")
print(f"\ttenemos un diametro de inductor de {Dind} cm.\n")

# # verificar efecto pelicular, Dcu/d > 5
# d=(6.62/(fo/1E6))*1E-2
# # print(Dcu/d)
# if Dcu/d <= 5: print("usar caño.\n".capitalize())
# else: print("usar alambre.\n".capitalize())

# criterio de separacion optima de Butterworth [vue/cm]
""" Nsop = 1/(1.41 * Dcu[cm])"""
Nsop=1/(1.41 * (Dcu/1E-2))

# ----------------------------------------------------------------------------------
# diseño, ecuacion K de nagaoka
print("-"*60)
print("Diseño, ecuacion K de Nagaoka:")
print("-"*60)

""" L[uH] = (Dind[cm])^3 * (Nsop[vue/cm])^2 * K * 1E-3 """

K=LT/(Dind**3 * Nsop**2 * 1E-3)
print(Dind,Nsop,LT)
print(f"Coeficiente K: {K} ({np.log10(K)} en log).\n")

# entrar a grafica K
relLD=1.12
print(f"Entrando al g:J/K para el K de Nagaoka, nos da la relacion l/Dind: {relLD} aprox.\n")

# largo del inductor teorico
l=relLD*Dind
# numero de espiras teorico
N=Nsop*l

print(f"tenemos un inductor de {l} cm de largo y {N} vueltas.\n".capitalize())

# correjir largo [cm] y numero de espiras
l=1.5
N=9
relLD=l/Dind

print(f"corrijiendo a {N} espiras y {l} cm de largo, tenemor la nueva relacion l/Dind {relLD}.\n".capitalize())

# verificacion, ecuacion J de Nagaoka
""" L[uH] = N^2 * (Dind [cm])/2 * J * 1E-3 """
print(f"{'-'*60} \nVerificacion, ecuacion J de Nagaoka: \n{'-'*60}")

J=9
L=(N**2) * Dind/2 * J * 1E-3
print(f"Entrando al g:J/K para la relacion l/Dind {relLD}, da un J de {J}.\nQuedando un inductor de {L} uH.\n")

# inductancia teorica
""" L=(uo * Aind * N^2)/l """
Aind=np.pi*((Dind*1E-2)/2)**2
print(f"inductancia teorica de: {( ((4*np.pi*1E-7)*Aind*N**2)/(l*1E-2) )/1E-6} uH")

# inductancia a H
L=L*1E-6

para inductor de 0.1 uH, con nucleo de 1.0 cm y cobre de 1.0 mm
	tenemos un diametro de inductor de 1.0999999999999999 cm.

------------------------------------------------------------
Diseño, ecuacion K de Nagaoka:
------------------------------------------------------------
1.0999999999999999 7.092198581560284 0.1
Coeficiente K: 1.493688955672427 (0.17426016983608478 en log).

Entrando al g:J/K para el K de Nagaoka, nos da la relacion l/Dind: 1.12 aprox.

Tenemos un inductor de 1.232 cm de largo y 8.73758865248227 vueltas.

Corrijiendo a 9 espiras y 1.5 cm de largo, tenemor la nueva relacion l/dind 1.3636363636363638.

------------------------------------------------------------ 
Verificacion, ecuacion J de Nagaoka: 
------------------------------------------------------------
Entrando al g:J/K para la relacion l/Dind 1.3636363636363638, da un J de 9.
Quedando un inductor de 0.40095 uH.

inductancia teorica de: 0.6448799515671788 uH


## Calcular R del inductor

In [11]:
# resistividad del cobre [ohm/m]
pcu=1.71E-8
# long de alambre aprox. [m]
lcu=Dind*N*1E-2
# seccion de cobre [m^2]
scu=np.pi*(Dcu/2)**2

# resitencia del inductor aprox.
""" R = (pcu * lcu)/Sn  """
r=pcu*lcu/scu

print(f"resistencia del inductor {r} Ohm ({r/1E-3} mOhm) aprox.\n")

resistencia del inductor 0.002155467225282157 Ohm (2.1554672252821567 mOhm) aprox.



## Correjir C

In [12]:
# ajustar nuevo valor de C
sp.pprint(ec0)

print(f"\ndespejando, CT = {sp.solve(ec0,ssCo)}")

""" 1/(4*pi^2 * L * fo^2 )"""
C=1/((4*np.pi**2)*L*fo**2)

print(f"\ncorrejido a C={C/1E-12} pF.\n")

               1        
ssfo = ─────────────────
             ___________
       2⋅π⋅╲╱ ssCo⋅ssLo 

despejando, CT = [1/(4*pi**2*ssLo*ssfo**2)]

correjido a C=246.78006833949496 pF.



## Calcular RT con factor Q

In [38]:
# factor de merito del inductor
""" Q = XLo/RT = fo/BW """
Q=fo/BW

RT=2*np.pi*fo*L/Q

print(f"para respetar BW, se necesita una RT de {RT} Ohm.\n")
print(Q,RT)

para respetar BW, se necesita una RT de 4.030789038261847 Ohm.

10.0 4.030789038261847


## Buscar relacion C2/C1

In [41]:
# transformando circuito y reflejando: Ra -> rrRa = Ra * (1+C2/C1)^2 = 2RT

""" 2*RT = Ra * (1+C2/C1)^2 """
ssRT,ssRa,ssC1,ssC2=sp.symbols("ssRT,ssRa,ssC1,ssC2")
ec1=sp.Eq(2*ssRT,ssRa*(1+ssC2/ssC1)**2)

sp.pprint(ec1)

# despejando relacion C1/C2
print("\ndespejando relacion C1/C2:")
print(sp.solve(ec1,(ssC2/ssC1)))

""" C2/C1 = sqrt(2*ssRT/ssRa) - 1 """
relC21=np.sqrt(2*RT/Ra) - 1

print(relC21)

                        2
              ⎛    ssC₂⎞ 
2⋅ssRT = ssRa⋅⎜1 + ────⎟ 
              ⎝    ssC₁⎠ 

despejando relacion C1/C2:
[(-ssRa + sqrt(2)*sqrt(ssRT*ssRa))/ssRa, -(ssRa + sqrt(2)*sqrt(ssRT*ssRa))/ssRa]
-0.5984634991305599


In [42]:
Primer relacion C2/C1 negativa.
2*RT/Ra < 1 -> subir RT -> subir LT -> bajar CT -> cambiar de 50 * 4pF a 20 * 4pF

SyntaxError: invalid syntax (3820261186.py, line 1)